In [17]:
import unidecode
import re
import pandas as pd
import datetime
from datetime import date
import json

In [6]:
import requests
from bs4 import BeautifulSoup

In [7]:
with open('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Congressional-dataMX/data/00-id_data/00-id/id_legislatura62.json') as d:
    data = json.load(d)

In [10]:
link_curricula = data[0]['link_curricula']

In [11]:
r = requests.get(link_curricula)
soup = BeautifulSoup(r.content, 'html.parser')

In [14]:
link_curricula

'http://sitllxii.diputados.gob.mx/curricula.php?dipt=430'

In [19]:
def get_link_to_periodos(link_curricula):
    r = requests.get(link_curricula)
    soup = BeautifulSoup(r.content, 'html.parser')
    for i in soup.findAll('a'):
        link = 'http://sitllxii.diputados.gob.mx/' + i.get('href')
        if any(re.findall('asistencias', link)):
            return(link)
        else:
            continue

In [20]:
get_link_to_periodos(link_curricula)

'http://sitllxii.diputados.gob.mx/asistencias_diputados_xperiodonplxii.php?dipt=430'

In [21]:
r = requests.get(get_link_to_periodos(link_curricula))
soup = BeautifulSoup(r.content, 'html.parser')

In [25]:
def get_links_periodos(link_curricula):
    r = requests.get(get_link_to_periodos(link_curricula))
    soup = BeautifulSoup(r.content, 'html.parser')
    links_periodos = []
    for i in soup.findAll('a'):
        link = 'http://sitllxii.diputados.gob.mx/' + i.get('href')
        if any(re.findall('asistencias', link)):
            links_periodos.append(link)
        else:
            continue
    return(links_periodos)

In [27]:
links_periodos = get_links_periodos(link_curricula)

In [28]:
links_periodos

['http://sitllxii.diputados.gob.mx/asistencias_por_pernplxii.php?iddipt=430&pert=1',
 'http://sitllxii.diputados.gob.mx/asistencias_por_pernplxii.php?iddipt=430&pert=3',
 'http://sitllxii.diputados.gob.mx/asistencias_por_pernplxii.php?iddipt=430&pert=13',
 'http://sitllxii.diputados.gob.mx/asistencias_por_pernplxii.php?iddipt=430&pert=5',
 'http://sitllxii.diputados.gob.mx/asistencias_por_pernplxii.php?iddipt=430&pert=7',
 'http://sitllxii.diputados.gob.mx/asistencias_por_pernplxii.php?iddipt=430&pert=15',
 'http://sitllxii.diputados.gob.mx/asistencias_por_pernplxii.php?iddipt=430&pert=16',
 'http://sitllxii.diputados.gob.mx/asistencias_por_pernplxii.php?iddipt=430&pert=17',
 'http://sitllxii.diputados.gob.mx/asistencias_por_pernplxii.php?iddipt=430&pert=18',
 'http://sitllxii.diputados.gob.mx/asistencias_por_pernplxii.php?iddipt=430&pert=9',
 'http://sitllxii.diputados.gob.mx/asistencias_por_pernplxii.php?iddipt=430&pert=11']

In [29]:
r = requests.get(links_periodos[0])
soup = BeautifulSoup(r.content, 'html.parser')

In [61]:
len(soup.findAll('table')[0].findAll('table')[4].findAll('tr'))

37

In [66]:
soup.findAll('table')[0].findAll('table')[4].findAll('tr')[0].findAll('tr')

36

In [127]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver import Chrome
from selenium import webdriver
import re
import time
import pandas as pd
import datetime
import uuid
from tqdm import tqdm
from datetime import date
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache


In [68]:
driver.get(links_periodos[0])

In [72]:
table = driver.find_element_by_css_selector('body > table > tbody > tr:nth-child(4) > td > table > tbody > tr:nth-child(2) > td > table:nth-child(2) > tbody')

In [79]:
calendar_tables = table.find_elements_by_css_selector('table')[:-1]

In [80]:
calendar_tables[0].

<selenium.webdriver.remote.webelement.WebElement (session="168a0e4cb7474f05b0628a7690f99e45", element="c9dd0775-bce6-4e0a-b607-d25142e42e7f")>

In [97]:
def get_month_table(inner_table):
    text_month = inner_table.find_element_by_class_name('TitulosVerde').text.lower()
    mes = month_to_number(re.findall('[a-z]+', text_month)[0])
    año = int(re.findall('[0-9]+', text_month)[0])
    return(mes, año)

In [113]:
calendar_tables[0].find_elements_by_css_selector('td')[6].text

'1\nA'

In [117]:
data[0]

{'legislatura': 62,
 'id_legislador': '111264d0-fe1c-11ea-95ca-acde48001122',
 'nombre_completo': 'Aceves y del Olmo Carlos Humberto',
 'tipo_legislador': 'Representacion proporcional',
 'entidad': 'Ciudad de Mexico',
 'distrito': '4',
 'curul': 'H-264',
 'suplente_propietario': 'P',
 'name_suplente': 'Juan Carlos Flores Flores',
 'partido': 'pri',
 'link_curricula': 'http://sitllxii.diputados.gob.mx/curricula.php?dipt=430'}

In [128]:

def get_asistencias_member(dict_member):
    list_dict = []
    
    links_periodos = get_links_periodos(dict_member['link_curricula'])
    for link in links_periodos:
        driver.get(link)
        table = driver.find_element_by_css_selector('body > table > tbody > tr:nth-child(4) > td > table > tbody > tr:nth-child(2) > td > table:nth-child(2) > tbody')
        calendar_tables = table.find_elements_by_css_selector('table')[:-1]
    
    
        for cal in calendar_tables:
            mes, año = get_month_table(cal)
            for i in cal.find_elements_by_css_selector('td'):
                text_cuadro = i.text
                if any(re.findall('\n', text_cuadro)):
                    day, tipo_asistencia = text_cuadro.split('\n')
                    temp = {}
                    temp['legislatura'] = 62
                    temp['year_session'] = año
                    temp['month_session'] = mes
                    temp['day_session'] = int(day)
                    temp['member'] = dict_member['nombre_completo']
                    temp['party'] = dict_member['partido']
                    temp['type_attendance'] = tipo_asistencia
                    temp['id'] = dict_member['id_legislador']
                    list_dict.append(temp)
                else:
                    continue
    return(list_dict)

In [124]:
from tqdm import tqdm

In [126]:
data_prop = []
for i in data:
    if i['suplente_propietario'] == 'P':
        data_prop.append(i)
    else:
        continue

In [129]:
all_asistencias = []
for i in tqdm(data_prop):
    list_asistencias = get_asistencias_member(i)
    for j in list_asistencias:
        all_asistencias.append(j)

100%|██████████| 500/500 [2:59:27<00:00, 21.53s/it]  


In [130]:
with open('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Congressional-dataMX/data/04-floor_attendance/02-id_members/floor_attend_dip62.json', 'w') as d:
    json.du

{'legislatura': 62,
 'year_session': 2012,
 'month_session': 9,
 'day_session': 1,
 'member': 'Aceves y del Olmo Carlos Humberto',
 'party': 'pri',
 'type_attendance': 'A',
 'id': '111264d0-fe1c-11ea-95ca-acde48001122'}

In [ ]:
if any(re.findall('\n', text_cuadro)):
    day, tipo_asistencia = text_cuadro.split('\n')
    print()

In [112]:
for i in calendar_tables[0].find_elements_by_css_selector('td'):
    text_cuadro = i.text
    if any(re.findall('\n', text_cuadro)):
        day, tipo_asistencia = text_cuadro.split('\n')
        print(day, ': ', tipo_asistencia)
    else:
        continue

1 :  A
4 :  A
6 :  A
11 :  A
13 :  A
18 :  A
20 :  A
25 :  A
27 :  PM
28 :  PM/A


In [87]:
re.findall('[a-z]+', text_month)[0]

'septiembre'

In [85]:
    text_month = calendar_tables[0].find_element_by_class_name('TitulosVerde').text.lower()

In [116]:
def month_to_number(m):
    if m == 'enero':
        return(1)
    elif m == 'febrero':
        return(2)
    elif m == 'marzo':
        return(3)
    elif m == 'abril':
        return(4)
    elif m == 'mayo':
        return(5)
    elif m == 'junio':
        return(6)
    elif m == 'julio':
        return(7)
    elif m == 'agosto':
        return(8)
    elif m == 'septiembre':
        return(9)
    elif m == 'octubre':
        return(10)
    elif m == 'noviembre':
        return(11)
    elif m == 'diciembre':
        return(12)

In [24]:
links_periodos

['http://sitllxii.diputados.gob.mx/asistencias_por_pernplxii.php?iddipt=430&pert=1',
 'http://sitllxii.diputados.gob.mx/asistencias_por_pernplxii.php?iddipt=430&pert=3',
 'http://sitllxii.diputados.gob.mx/asistencias_por_pernplxii.php?iddipt=430&pert=13',
 'http://sitllxii.diputados.gob.mx/asistencias_por_pernplxii.php?iddipt=430&pert=5',
 'http://sitllxii.diputados.gob.mx/asistencias_por_pernplxii.php?iddipt=430&pert=7',
 'http://sitllxii.diputados.gob.mx/asistencias_por_pernplxii.php?iddipt=430&pert=15',
 'http://sitllxii.diputados.gob.mx/asistencias_por_pernplxii.php?iddipt=430&pert=16',
 'http://sitllxii.diputados.gob.mx/asistencias_por_pernplxii.php?iddipt=430&pert=17',
 'http://sitllxii.diputados.gob.mx/asistencias_por_pernplxii.php?iddipt=430&pert=18',
 'http://sitllxii.diputados.gob.mx/asistencias_por_pernplxii.php?iddipt=430&pert=9',
 'http://sitllxii.diputados.gob.mx/asistencias_por_pernplxii.php?iddipt=430&pert=11']

In [ ]:
links_periodos = []

In [ ]:
http://sitllxii.diputados.gob.mx/asistencias_diputados_xperiodonplxii.php?dipt=430

In [4]:
link_members = 'http://sitllxii.diputados.gob.mx/listado_diputados_gpnp.php?tipot=TOTAL'

In [5]:
r = requests.get(link_members)
soup = BeautifulSoup(r.content, 'html.parser')

In [ ]:
soup = BeautifulSoup(r.content, 'html.parser')
divs_content = soup.findAll("article", {"class": "nota"})